In [1]:
import pandas as pd
import numpy as np
import pickle # sirve para guardar cualquier objeto binario, inclusi el pipline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
import os
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, mean_absolute_percentage_error

In [2]:
data = pd.read_csv("../data/wines_dataset.csv", sep="|")
data.columns= ["fixed_acidity", "volatile_acidity", "citric_acid", "residual_sugar", "chlorides", "free_sulfur_dioxide", "total_sulfur_dioxide", "density", "pH", "sulphates", "alcohol", "quality", "class_"]
data.columns

Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'class_'],
      dtype='object')

In [3]:
# Cargar el archivo CSV

"""
División de datos en entrenamiento y prueba.

Se toma el 80% de los datos para entrenamiento y el 20% restante para prueba.

Parámetros:
- test_size (float): Proporción del conjunto de prueba (0.2 = 20% de los datos).
- random_state (int): Semilla para la reproducibilidad de la división.

Salida:
- train (DataFrame): Conjunto de entrenamiento.
- test (DataFrame): Conjunto de prueba.
"""
train_1, train_2 = train_test_split(data, test_size=0.2, random_state=42)
train, test = train_test_split(train_1, test_size=0.2, random_state=42)
"""
Guardado de los conjuntos de datos en archivos CSV.

Los archivos se almacenan en la carpeta "./data/" con los nombres:
- wines_train.csv → Contiene el 80% de los datos para entrenamiento.
- wines_test.csv → Contiene el 20% de los datos para prueba.

index=False evita que se guarde el índice en los archivos.
"""
# Crear el directorio si no existe

train.to_csv(os.path.join("../data/", 'wines_train.csv'), index=False)
test.to_csv(os.path.join("../data/", 'wines_test.csv'), index=False)
train_2.to_csv(os.path.join("../data/", 'wines_retrain.csv'), index=False)

In [4]:
# Cargar el conjunto de datos de entrenamiento desde el archivo CSV
df_train = pd.read_csv("../data/wines_train.csv")
df_test = pd.read_csv("../data/wines_test.csv")

In [5]:
df_train.columns

Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'class_'],
      dtype='object')

In [6]:
# target_reg = "alcohol" → Problema de Regresión
#    - La variable `alcohol` indica el porcentaje de alcohol en el vino.
#    - Es una variable numérica continua.
#    - Se utilizarán modelos de regresión, como RandomForestRegressor o LinearRegression.
#    - Requiere escalado de las variables numéricas (StandardScaler) y codificación de las categóricas (OneHotEncoder).

target_reg = "alcohol"
y_train_reg = df_train["alcohol"]
y_test_reg = df_test["alcohol"]


# OPCIÓN REGRESIÓN


In [7]:
# Definición de características (features) para el problema de regresión:

# features_cat_reg → Variables categóricas
#    - Contiene "class" (tipo de vino: tinto o blanco) y "quality" (calidad del vino).
#    - Estas variables deben ser transformadas con OneHotEncoder.

# features_num_reg → Variables numéricas
#    - Se seleccionan todas las columnas del DataFrame excepto las categóricas.
#    - Esto se logra con una lista por comprensión, excluyendo las columnas en features_cat_reg.
#    - Estas variables deben ser escaladas con StandardScaler.

features_cat_reg = ["class_", "quality"]
features_num_reg = [col for col in df_train.columns if col not in features_cat_reg]

# Ahora features_num_reg contiene solo las variables numéricas.

In [8]:
# Estudio de correlaciones entre las variables numéricas y la variable objetivo de regresión.

# Calculamos la matriz de correlación solo para las variables numéricas.
#    - Se utiliza df[features_num_reg] para excluir las variables categóricas.
#    - numeric_only="True" se usa para evitar advertencias en versiones recientes de pandas.

# Extraemos la correlación absoluta de cada variable con el target de regresión ("alcohol").
#    - np.abs() se usa para obtener la magnitud de la correlación sin importar el signo.
#    - .sort_values(ascending=False) ordena las variables de mayor a menor correlación.

corr = df_train[features_num_reg].corr(numeric_only="True")
serie_corr = np.abs(corr[target_reg]).sort_values(ascending=False)
serie_corr

# Ahora, serie_corr contiene las variables ordenadas según su grado de correlación con "alcohol".

alcohol                 1.000000
density                 0.673527
residual_sugar          0.355085
total_sulfur_dioxide    0.274704
chlorides               0.266000
free_sulfur_dioxide     0.183075
pH                      0.104613
fixed_acidity           0.082380
volatile_acidity        0.036412
citric_acid             0.007774
sulphates               0.001449
Name: alcohol, dtype: float64

In [9]:
# Selección de características numéricas basadas en la correlación con el target de regresión.

# Definimos un umbral mínimo de correlación.
#    - r_min = 0.05 significa que solo seleccionaremos variables con correlación mayor a 0.05.

r_min = 0.05

# Seleccionamos las variables numéricas con correlación significativa con el target.
#    - Tomamos solo las variables con correlación absoluta mayor que r_min.
#    - Convertimos los nombres de las columnas en una lista.
#    - Eliminamos el target ("alcohol") de la lista, ya que no es una feature.

features_num_reg_1 = serie_corr[serie_corr > r_min].index.to_list()
features_num_reg_1.remove(target_reg)

# Identificamos las variables numéricas con correlación baja o nula con el target.
#    - Excluimos las variables en features_num_reg_1.
#    - Excluimos el target y las variables categóricas.

features_num_reg_2 = [col for col in df_train.columns if col not in features_num_reg_1 and col != target_reg
                       and col not in features_cat_reg]

# Ahora:
# - features_num_reg_1 contiene las variables más correlacionadas con "alcohol".
# - features_num_reg_2 contiene las variables menos correlacionadas.

In [10]:
features_num_reg_1

['density',
 'residual_sugar',
 'total_sulfur_dioxide',
 'chlorides',
 'free_sulfur_dioxide',
 'pH',
 'fixed_acidity']

In [11]:
# Definición del problema de regresión.

# target_reg → Variable objetivo para regresión.
#    - Se ha definido previamente como "alcohol".
#    - Representa el porcentaje de alcohol en el vino (variable continua).
#    - Es un problema de regresión, ya que el target es numérico y continuo.

target_reg

'alcohol'

In [12]:
# Definición de las características categóricas para el problema de regresión.

# features_cat_reg → Variables categóricas a incluir en el modelo de regresión.
#    - Contiene:
#      - "class" (tipo de vino: tinto o blanco).
#      - "quality" (calidad del vino en una escala categórica).
#    - Ambas variables deben ser transformadas con OneHotEncoder para su uso en modelos numéricos.

features_cat_reg

['class_', 'quality']

In [13]:
# Definición de características numéricas principales para regresión.

# features_num_reg_1 → Variables numéricas con mayor correlación con el target de regresión ("alcohol").
#    - Se han seleccionado aquellas con correlación absoluta > r_min (0.05).
#    - Estas variables son las más relevantes para predecir el contenido de alcohol en el vino.
#    - Se recomienda aplicar StandardScaler para normalizar estas características antes del modelado.

features_num_reg_1

['density',
 'residual_sugar',
 'total_sulfur_dioxide',
 'chlorides',
 'free_sulfur_dioxide',
 'pH',
 'fixed_acidity']

In [14]:
# Definición de columnas a incluir y excluir en el modelo de regresión.

# columns_to_keep_reg → Columnas que se mantendrán en el modelo de regresión.
#    - Incluye:
#      - features_num_reg_1 (variables numéricas más correlacionadas con el target).
#      - features_cat_reg (variables categóricas a transformar con OneHotEncoder).

# columns_to_exclude_reg → Columnas que se excluirán del modelo de regresión.
#    - Se obtienen eliminando de df.columns las variables incluidas en columns_to_keep_reg.
#    - Estas columnas no serán utilizadas en el modelo.

columns_to_keep_reg = features_num_reg_1 + features_cat_reg

columns_to_exclude_reg = [col for col in df_train.columns if col not in columns_to_keep_reg]

columns_to_exclude_reg

['volatile_acidity', 'citric_acid', 'sulphates', 'alcohol']

In [15]:
# Definición de Pipelines para preprocesamiento de datos en regresión.

# cat_pipeline → Preprocesamiento de variables categóricas.
#    - "Impute_Mode": Imputa valores faltantes con la moda (valor más frecuente).
#    - "OHEncoder": Aplica OneHotEncoder, ignorando categorías desconocidas.

# logaritmica → Transformación logarítmica de variables numéricas.
#    - Usa FunctionTransformer con np.log1p para estabilizar distribuciones sesgadas.
#    - feature_names_out="one-to-one" mantiene los nombres originales de las características.

# num_pipeline → Preprocesamiento de variables numéricas.
#    - "Impute_Mean": Imputa valores faltantes con la media.
#    - "logaritmo": Aplica la transformación logarítmica definida antes.
#    - "SScaler": Aplica StandardScaler para normalizar las variables numéricas.

# imputer_step_reg → ColumnTransformer para aplicar los Pipelines según el tipo de variable.
#    - "Process_Numeric": Aplica num_pipeline a features_num_reg_1 (variables numéricas seleccionadas para regresión).
#    - "Process_Categorical": Aplica cat_pipeline a features_cat_reg (variables categóricas seleccionadas para regresión).
#    - "Exclude": Elimina las columnas en columns_to_exclude_reg.
#    - remainder="passthrough": Mantiene cualquier otra columna sin modificar.

# pipe_missings_reg → Pipeline final que aplica el ColumnTransformer imputer_step_reg.

cat_pipeline = Pipeline(
    [("Impute_Mode", SimpleImputer(strategy="most_frequent")),  # Imputación con la moda
     ("OHEncoder", OneHotEncoder(handle_unknown='ignore'))  # Manejar categorías desconocidas
    ]
)

logaritmica = FunctionTransformer(np.log1p, feature_names_out="one-to-one") 
# Esto le indica al Pipeline que el número de características no cambia y que puede usar los nombres originales.

num_pipeline = Pipeline(
    [("Impute_Mean", SimpleImputer(strategy = "mean")), # prevision que en el futuro lleguen datos faltantes
     ("logaritmo", logaritmica),
     ("SScaler", StandardScaler()),
    ]
)

imputer_step_reg = ColumnTransformer(
    [("Process_Numeric", num_pipeline,features_num_reg_1), # feature_numericas seleccionadas para clasificación
     ("Process_Categorical", cat_pipeline, features_cat_reg), # feature_categoriacas seleccionadas para regresión
     ("Exclude", "drop", columns_to_exclude_reg)
    ], remainder = "passthrough"
    )

pipe_missings_reg = Pipeline([("first_stage", imputer_step_reg)])

In [16]:
### EVALUACION DE MODELOS DE REGRESION
# Definimos los modelos a analizar en el problema de regresión.
#   - RandomForestRegressor
#  - XGBRegressor
# - LGBMRegressor

In [17]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [18]:
# Definición de Pipelines para modelos de regresión.

# XGB_pipeline → Pipeline para XGBRegressor.
#    - "Preprocesado": Aplica el Pipeline de preprocesamiento pipe_missings_reg.
#    - "Modelo": Implementa XGBRegressor, basado en árboles de decisión con boosting.


# Validación cruzada con 5 folds.
#    - Se evalúan los tres modelos utilizando cross_val_score.
#    - La métrica utilizada es "neg_mean_absolute_percentage_error" (MAPE negativo).
#    - Se imprime el promedio del error absoluto porcentual y los valores individuales por fold.


XGB_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_reg),  
     ("Modelo", XGBRegressor(learning_rate= 0.1, max_depth= 8, n_estimators= 400))  # Modelo de boosting basado en XGBoost
    ])

resultado_reg = cross_val_score(XGB_pipeline, df_train, y_train_reg, cv=5, scoring="neg_mean_absolute_percentage_error")
XGB_pipeline.fit(df_train,y_train_reg)
y_pred=XGB_pipeline.predict(df_test)
mape_test=mean_absolute_percentage_error(y_pred=y_pred,y_true=y_test_reg)
print(f"XGB cross_val media_mape: {np.mean(-resultado_reg):.4f}")  # Se invierte el signo para interpretar el MAPE positivo
print(f"XGB test mape: {mape_test}")

XGB cross_val media_mape: 0.0258
XGB test mape: 0.024226998823328058


In [19]:
# Selección del mejor modelo tras la búsqueda de hiperparámetros con GridSearchCV.
#
# - Se toma el modelo con el mejor score obtenido en la validación cruzada.
# - Se extrae el nombre del mejor modelo desde best_grids_reg.
# - Se usa ese nombre para acceder al mejor GridSearchCV almacenado en pipe_grids_reg.
# - El modelo seleccionado se almacenará en best_model_reg para futuras predicciones.

# Extraer el mejor modelo basado en la mejor puntuación obtenida.
# best_model_reg = pipe_grids_reg[best_grids_reg.iloc[0, 0]]

# Mostrar el mejor modelo seleccionado
XGB_pipeline

Pipeline(steps=[('Preprocesado',
                 Pipeline(steps=[('first_stage',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('Process_Numeric',
                                                                   Pipeline(steps=[('Impute_Mean',
                                                                                    SimpleImputer()),
                                                                                   ('logaritmo',
                                                                                    FunctionTransformer(feature_names_out='one-to-one',
                                                                                                        func=<ufunc 'log1p'>)),
                                                                                   ('SScaler',
                                                                                    StandardScaler())]),
                                                                   ['density',
                                                                    'residual_sugar',
                                                                    'total_sulfur_dioxi...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=8, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=400, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [20]:
mape_test

0.024226998823328058

In [21]:
df_test.columns

Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'class_'],
      dtype='object')

In [22]:
# Guardado del mejor modelo de regresión usando Pickle.
#
# - Se asegura de que el directorio "src/models" exista antes de guardar el modelo.
# - Serializa el mejor modelo encontrado (`best_model_reg`) en un archivo .pkl.
# - El modelo guardado podrá ser cargado posteriormente sin necesidad de volver a entrenarlo.

# Asegurar que el directorio "src/models" exista antes de guardar el modelo.

# Guardar el mejor modelo en un archivo .pkl dentro de "models".
with open('../modelo_pipeline_reg.pkl', 'wb') as archivo:
    pickle.dump(XGB_pipeline, archivo)

In [25]:
data = pd.DataFrame([{
    'fixed_acidity': 6.6,
    'volatile_acidity': 0.16,
    'citric_acid': 0.3,
    'residual_sugar': 1.6,
    'chlorides': 0.034,
    'free_sulfur_dioxide': 15.0,
    'total_sulfur_dioxide': 78.0,
    'density': 0.992,
    'pH': 3.38,
    'sulphates': 0.44,
    'alcohol': 11.2,
    'quality': 6,
    'class_': 'white'
}])

# Asegúrate de que el modelo y pipeline están cargados
# Suponiendo que ya tienes:
# from joblib import load
# XGB_pipeline = load('XGB_pipeline.joblib')

# Hacer la predicción
prediction = XGB_pipeline.predict(data)

In [26]:
prediction

array([10.656982], dtype=float32)

In [23]:
df_test.iloc[0,:]

fixed_acidity             6.6
volatile_acidity         0.16
citric_acid               0.3
residual_sugar            1.6
chlorides               0.034
free_sulfur_dioxide      15.0
total_sulfur_dioxide     78.0
density                 0.992
pH                       3.38
sulphates                0.44
alcohol                  11.2
quality                     6
class_                  white
Name: 0, dtype: object